## Exemplary script for unconstrained simultaneous retrieval

This Python notebook provides an exemplary script for the unconstrained simultaneous retrieval of sea ice thickness d and sea ice Salinity d. Brightness temperature (TB) simulation is computed with SMRT. Therefore, to run the script it is required to have SMRT installed. I had SMRT installed on the server of University of Hamburg. Hence, if working here, simply run

-> module load anaconda3

in your terminal prior to executing this script. When working from your home PC, you must install SMRT first. Therefore, you can use

-> git clone https://github.com/smrt-model/smrt



Please make sure you set the Pythonpath for SMRT in your /.bash_rc. Therefore, edit your /.bash_rc with

-> export PYTHONPATH="/home/User/location/smrt"


For a more detailed description on how to install SMRT, see https://github.com/smrt-model.




For more information in the below script, have a look in my Thesis $\textit{Advancing sea ice remote sensing through wideband brightness temperature simulation}$ 

or feel free to contact me: simon.we@me.com


This code follows the basic idea and snippet of Dr. Lars Kaleschke, Alfred Wegener Institut, Bremerhaven. lars.kaleschke@awi.de

Note: TB simulation with SMRT is likely to cause multi-threading. You can avoid multi-threading by implementing 


in your code.

When running the script for the first time, make sure to chose low N and low NI to keep computational time at a minimum

In [1]:
%pylab inline
import scipy.stats as stats
import pandas as pd
from io import StringIO
from scipy.optimize import curve_fit, minimize
import scipy.io as sio
import numpy as np
import matplotlib.pyplot as plt

#load SMRT modules
from smrt import make_model, make_snowpack, sensor_list, make_ice_column
from smrt.utils import dB
from smrt import PSU
from smrt.permittivity.wetsnow import wetsnow_permittivity

Populating the interactive namespace from numpy and matplotlib


In [2]:
########################## TB simulation #############################################################################

#TB in dependency of d and S
def Tb_S_d(p):
    d,S_ice=p[0],p[1]
    T_ice = linspace(273.15+(temperature_init), 273.15 - 1.8, 2*set_layers+1)
    c=1.0
    return Tb_f(c,d,T_ice,S_ice)

#TB simulation with SMRT for frequency f
def Tb_f(c,d,T_ice,S_ice):
    Tb=[]
    for f in frequency:
        sensor = sensor_list.passive(f*1e9, incidence_angle) #sensor configuration
        temperature = linspace(273.15+(surface_temperature), 273.15+(water_temperature), 2*set_layers+1)
                                #temperature gradient in the ice from surface temp
                                #at top to temperature of water at bottom
        temperature = temperature[1::2] # average temperatures of layers (=temperature at midpoints)
        thickness = array([d/set_layers] * set_layers) #thickness of each layer
        salinity = S_ice*PSU #PSU as SI unit for SMRT
        
        #modelling of a sea ice column with the make layer submodel
        ice_column = make_ice_column(ice_type=ice_type,thickness=thickness,
                                temperature=temperature,
                                microstructure_model="exponential",
                                brine_inclusion_shape=inclusion_shape, 
                                salinity=salinity,
                                corr_length=corr_length,
                                add_water_substrate="ocean"
                                )

        # run the model
        res = m.run(sensor, ice_column)
        res_TB = (res.TbH() + res.TbV())/2 #mean TB from vertically and horizontally polarized TB
        Tb.append(res_TB)
    return array(Tb)

In [ ]:
# objective func
def objective_func(p, noise):
    Tb_model=Tb_S_d(p)
    return (Tb_model-noise)**2 

In [ ]:
Gtol = 1e-3 #stopping criteria
N_TB_f = 2 # number of frequencies considered in the retrieval

In [ ]:
###########################  SMRT setting  #################################################

#medium settings
ice_type = 'firstyear' # first-year or multi-year sea ice
porosity = 0. # ice porosity in fractions, [0..1]
surface_temperature = -20 
water_temperature = -1.8
corr_length = 1.0e-3 #correlation length
inclusion_shape={"spheres":1,"random_needles":0} #set ratio of spheres and needles

#create the sensor
incidence_angle = 30 #angle of incidence
n_max_stream = 32 #number of computed streams
frequency = linspace(0.5,4,N_TB_f) #frequencies in GHz
len_frequency = len(frequency)
m = make_model("iba", "dort", rtsolver_options ={"n_max_stream": n_max_stream}) #define the model

In [ ]:
###################### settings for the retrieval ###################################################################

bounds = ((0.02,1),(1,25)) #d bounds and S bounds
d_range= linspace(0.02,1,1*FI+1) #value range of investigated sea ice thickness
S_range= linspace(1,25,1*FI+1) #value range of investigated sea ice salinity
layers = 5 #np.round(np.linspace(2,10,len(d_range))).astype(int) #number of ice layers predefined
N = 10 # number of times retrieval of each (d,S) is repeated - for statistics (mean and std)

In [ ]:
RM_unrestricted = np.zeros((len(d_range),len(S_range),4))
    #array, which will be filled with re

for di,d in enumerate(d_range):
    d_try=d #initial thickness guess
    set_layers = layers #chose number of layers depending on sea ice thickness
    for si,s in enumerate(S_range):
        S_try = s 
        p_try=(d_try,S_ice_try)
        retrieval=[] #refresh retrieval array
        for i in range(N):# retrieval is repeated N times for stats
            p_try        = (d_try,S_try) # initial value (d,S)
            Tb_obs       = Tb_S_d(p_try) # simulate observed Tb
            Tb_obs_noise = Tb_obs + scipy.randn(len_frequency)*0.5 # Simulate measurement noise
            initial_guess = array([0.5,6.0]) # initial guess (d,S): starting point for the retrieval
            result = opti.least_squares(my_cost, initial_guess[:], 
                                        args = (Tb_obs_noise,),
                                        gtol=Gtol, 
                                        bounds=bounds)
            retrieval.append(result.x)
            
        retrieval=array(retrieval)
        d_retrieval,d_sig_retrieval=mean(retrieval[:,0]),std(d_try-retrieval[:,0])
        S_retrival,S_sig_retrieval=mean(retrieval[:,1]),std(S_ice_try-retrieval[:,1])
        RM_unrestricted[di,si,0]=d_try-d_retrival
        RM_unrestricted[di,si,1]=d_sig_retrieval
        RM_unrestricted[di,si,2]=S_ice_try-S_retrieval
        RM_unrestricted[di,si,3]=S_sig_retrieval

In [ ]:
############################# plot result ####################################################################

results_fig = figure(figsize=(16,12))
subplot(2,2,1)
pcolormesh(S_range,d_range,RM_unrestricted[:,:,0], cmap=cm.coolwarm,vmin=-0.3,vmax=0.3)
xlabel('S')
ylabel('d [m]')
colorbar()
title('$\Delta$d [m]')

subplot(2,2,2)
pcolormesh(S_range,d_range,RM_unrestricted[:,:,1], cmap=cm.hot_r,vmin=0,vmax=0.3)
xlabel('S')
ylabel('d [m]')
title('$\sigma_d$ [m]')
colorbar()

subplot(2,2,3)
pcolormesh(S_range,d_range,RM_unrestricted[:,:,2], cmap=cm.coolwarm,vmin=-3,vmax=3)
xlabel('S')
ylabel('d [m]')
colorbar()
title('$\Delta$S [g/kg]')
subplot(2,2,4)

pcolormesh(S_range,d_range,RM_unrestricted[:,:,3], cmap=cm.hot_r,vmin=0,vmax=3)
xlabel('S')
ylabel('d [m]')
title('$\sigma_s$ [g/kg]')
colorbar()